In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime
%matplotlib inline
import seaborn as sns
sns.set()
from matplotlib.backends.backend_pdf import PdfPages
#from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm
import matplotlib.colors as col
import matplotlib as mpl
import numpy as np #import for transposing
import math
import pylab as pl
import math
import import_ipynb
from sklearn.metrics import mean_squared_error,mean_absolute_error
from function_log import add_tdate,get_week,density_interp,extract_date,qtr_comp,IC_run_sim,new_compare,stoch_model,run_sim

from scipy.optimize import minimize

importing Jupyter notebook from function_log.ipynb


In [2]:
def find(condition):
    res, = np.nonzero(np.ravel(condition))
    return res

In [3]:
new_census = pd.read_csv('new_data.csv')
new_census['Date'] = pd.to_datetime(new_census['Date'])
new_census['ObsYear'] = new_census.Date.apply(lambda x: x.year)
new_census['ObsMonth'] = new_census.Date.apply(lambda x:x.month)

new_census.T = pd.to_numeric(new_census['T'],errors='coerce')
new_census.J = pd.to_numeric(new_census.J)
new_census.A = pd.to_numeric(new_census.A)
new_census.S = pd.to_numeric(new_census.S)

In [4]:
t_types = pd.read_csv('troop_types.csv')

rural_troops = list(t_types.loc[t_types['Type Code'] == 0]['Troop'])
rural = new_census.loc[new_census.Troop.isin(rural_troops)]

urb_troops = list(t_types.loc[t_types['Type Code'] == 1]['Troop'])
urban = new_census.loc[new_census.Troop.isin(urb_troops)]

In [5]:
from __future__ import print_function
import sys
import threading
from time import sleep
try:
    import thread
except ImportError:
    import _thread as thread

def quit_function():
        # print to stderr, unbuffered in Python 2.
        print('Parameter set rejected', file=sys.stderr)
        sys.stderr.flush() # Python 3 stderr is likely buffered.
        thread.interrupt_main() # raises KeyboardInterrupt
        

def exit_after(s):
    '''
    use as decorator to exit process if 
    function takes longer than s seconds
    '''
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(s, quit_function, args=[])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return inner
    return outer

##Initialize timeout function
@exit_after(300)
def min_func(paramset):
    
    with open('paramset_rural.txt', 'a') as param:
        param.write('{params}\n'.format(params=paramset))

        
    try:    
        ##Extract median NRMSE value from model run
        res_df = new_compare(rural,iters=25,
                             fus_fis=paramset[0],
                             no_ev=paramset[1],
                             rt_chg=paramset[2],
                             b_j=paramset[3],
                             d_j=paramset[4],
                             d_s=paramset[5],
                             d_a=paramset[6],
                             ae_j=paramset[7],
                             ae_s=paramset[8],
                             ae_a=paramset[9],
                             t_p=paramset[10],
                             por=paramset[11],
                             fsr=paramset[12],
                             thresh=paramset[13],
                             fis_thresh=paramset[14],
                             lit_size=paramset[15],
                             dfe='off'
                            )
        smape = res_df.loc[res_df.sMAPE<=100]
        smape = smape.loc[:,'sMAPE'].dropna()

        total = len(smape)
        good = len(smape.loc[smape <= 15])
        perc = 1-(good/total)
        
        print('Percent of High Error Runs: {}\n'.format(perc))
        with open('percenterr_rural.txt', 'a') as percenterr:
            percenterr.write('{percents}\n'.format(percents=perc))
        
        
        return perc
    
    except:
        return 1

In [ ]:
##USE PYSWARM HERE
from pyswarm import pso

#x0 = [0.4,0.8,0.1,0.299,0.345,0.216,0.143,0.5,0.5,0.5,0.3,0.2,0.4,4,45,4]

lb = [0.1,.8,.05,.254,.278,.136,.034,.1,.1,.1,.15,.1,.2,4,38,3]
ub = [0.7,.95,.5,.344,.412,.296,.252,.8,.8,.8,.70,.6,.8,6,68,5]

xopt,fopt = pso(min_func,lb,ub,maxiter=25,minstep=0.001,debug=True,minfunc=0.001)

No constraints given.


<string>:199: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: divide by zer

2021-03-08 14:57:34.817671 calculated new_compare
Percent of High Error Runs: 0.8124590163934426



<string>:199: RuntimeWarning: divide by zero encountered in double_scalars
<string>:199: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: divide by ze

2021-03-08 14:58:18.651439 calculated new_compare
Percent of High Error Runs: 0.7674897119341564



...

2021-03-08 15:18:05.342362 calculated new_compare
Percent of High Error Runs: 0.6858678955453149



<string>:199: RuntimeWarning: divide by zero encountered in double_scalars
<string>:199: RuntimeWarning: divide by zero encountered in double_scalars
Parameter set rejected
<string>:199: RuntimeWarning: divide by zero encountered in double_scalars
<string>:199: RuntimeWarning: divide by zero encountered in double_scalars
<string>:199: RuntimeWarning: divide by zero encountered in double_scalars
<string>:199: RuntimeWarning: divide by zero encountered in double_scalars
<string>:199: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: R

2021-03-08 15:23:49.408195 calculated new_compare
Percent of High Error Runs: 0.8198799199466311



<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:63: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encou

2021-03-08 15:24:36.875242 calculated new_compare
Percent of High Error Runs: 0.7639836289222374



In [ ]:
xopt
with open('optimiz_rural1.txt', 'a') as optimiz:
        optimiz.write('{params}\n'.format(params=xopt))

In [ ]:
fopt